Data Processing

In [ ]:

import pandas as pd
df = pd.read_csv('/work/final_forreal.csv')

home_ohe = pd.get_dummies(df['homeType'])
home_ohe
df = pd.concat([df, home_ohe], axis = 1)
df

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
#columns=['Lead', 'Address', 'city', 'cityId', 'county', 'countyFIPS', 'homeType', 'livingArea', 'livingAreaValue', 'parcelId', 'price', 'regionString', 'state', 'stateId', 'zipcode']

X = df.drop(columns=['Lead', 'Address', 'city', 'cityId', 'county', 'countyFIPS', 'homeType', 'livingArea', 'livingAreaValue', 'parcelId', 'price', 'regionString', 'state', 'stateId', 'zipcode'])
y = df['Lead']

# Replace NaN values with the mean of the non-missing values
imputer = SimpleImputer(strategy="mean")
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Use SMOTE to oversample the positive class to 30,000 samples
smote = SMOTE(sampling_strategy={1: 50000}, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

Model

In [ ]:
# Create a logistic regression model
model = LogisticRegression()

# Train the model on the training data
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate False Negative Rate
fnr = 1 - recall_score(y_test, y_pred)
print("False Negative Rate:", fnr)

# Calculate AUROC score
auroc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print("AUROC Score:", auroc)

# Create a confusion matrix
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion)

# Generate a classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)